#### Azure Batch Shipyard devito 

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
DOTENV_FILE_PATH = ['.', 'not_shared'] + ['general.env']
SIBLING_DOCKER_DOTENV_FILE_PATH = ['.', 'not_shared'] + ['sibling_docker.env']

docker_file_location = 'docker_files'
docker_build_location = 'docker_build'
docker_build_no_cache = ''  # '--no-cache' # or '' #

control_plane_docker_image_name = "azure_batch_shipyard_devito"
control_plane_docker_image_tag = ""

azure_batch_shipyard_devito_base_image_name = 'devito_azure_base:v1.0'
azure_batch_shipyard_devito_tti_image_name = 'devito_azure_tti:v1.0'

use_ACR_and_dockerhub = True

In [3]:
import sys, os, pathlib
import platform,  dotenv
import subprocess

In [4]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1067-azure-x86_64-with-debian-10.1'

'/workspace/notebooks'

In [5]:
def create_dot_env_file(dotenv_file_path_list):
    created_dotenv_file_path = os.path.join(*(dotenv_file_path_list)) 
    os.makedirs(os.path.join(*(dotenv_file_path_list[:-1])), exist_ok=True)
    pathlib.Path(created_dotenv_file_path).touch()
    return created_dotenv_file_path

dotenv_file_path = create_dot_env_file(DOTENV_FILE_PATH)
sibling_docker_dotenv_file_path = create_dot_env_file(SIBLING_DOCKER_DOTENV_FILE_PATH)

# # show .env file path
# !pwd
dotenv_file_path
sibling_docker_dotenv_file_path

'./not_shared/general.env'

'./not_shared/sibling_docker.env'

Save your sensitive info

In [6]:
ACR_NAME="fwi01acr"
ACR_PASSWORD=""
ACR_USERNAME="fwi01acr"

dockerhub_login = "georgedockeraccount"
dockerhub_pwd = ""

SUBSCRIPTION_ID=""

docker_container_mount_point = os.getcwd()

In [7]:
# dotenv.set_key(dotenv_file_path, 'ACR_NAME', ACR_NAME)
# dotenv.set_key(dotenv_file_path, 'ACR_PASSWORD', ACR_PASSWORD)
# dotenv.set_key(dotenv_file_path, 'ACR_USERNAME', ACR_USERNAME)

# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_LOGIN', dockerhub_login)
# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_PWD', dockerhub_pwd)

# dotenv.set_key(dotenv_file_path, 'SUBSCRIPTION_ID', SUBSCRIPTION_ID)

# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_name', control_plane_docker_image_name)
# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_tag', control_plane_docker_image_tag)

# dotenv.set_key(sibling_docker_dotenv_file_path, 'DOCKER_CONTAINER_MOUNT_POINT', docker_container_mount_point)

In [8]:
dotenv.load_dotenv(dotenv_path=dotenv_file_path)
docker_file_location = os.path.join(*(os.getcwd(), docker_file_location))

docker_file_name = 'Dockerfile'+ '_' + os.getenv('control_plane_docker_image_name')

docker_image_name_only = os.getenv('control_plane_docker_image_name')
image_version = os.getenv('control_plane_docker_image_tag')
if image_version!="":
    docker_file_name = docker_file_name +'_'+ image_version
    docker_image_name_only = docker_image_name_only +':'+ image_version

# docker_repo_name = os.getenv('DOCKERHUB_LOGIN') # or  
docker_repo_name = os.getenv('ACR_NAME')+'.azurecr.io' 
docker_image_name = docker_repo_name + '/' + docker_image_name_only

docker_file_dir = os.path.join(*([os.getcwd(), docker_file_location]))
os.makedirs(docker_file_dir, exist_ok=True)
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))

docker_build_dir = os.path.join(*([os.getcwd(), docker_build_location]))
os.makedirs(docker_build_dir, exist_ok=True)


docker_image_name
docker_file_dir
docker_file_path
docker_build_dir

True

'fwi01acr.azurecr.io/azure_batch_shipyard_devito'

'/workspace/notebooks/docker_files'

'/workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito'

'/workspace/notebooks/docker_build'

Dockerfile references:  
https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile  
https://github.com/matsuu/docker-azure-cli/blob/master/Dockerfile  
https://github.com/Azure/azure-cli/blob/dev/Dockerfile  
https://github.com/Azure/azure-functions-core-tools#other-linux-distributions  

In [9]:
%%writefile $docker_file_path 

# https://hub.docker.com/r/microsoft/azure-cli/dockerfile   
FROM debian:latest 
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile
# https://github.com/Azure/batch-shipyard/blob/master/cargo/Dockerfile
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    python3 python3-pip python3-setuptools \
    ca-certificates curl apt-transport-https lsb-release gnupg \
    zip unzip \
    wget rsync git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*   && \
    git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard && \
    cd /opt/batch-shipyard && \
    rm -rf .git .github .vsts && \
    rm -f .git* .travis.yml *.yml install*  && \
    python3 -m pip install --no-cache-dir --upgrade pip  && \
    pip3 install --no-cache-dir -r requirements.txt  && \
    pip3 install --no-cache-dir --no-deps -r req_nodeps.txt && \
    python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy 
# ENTRYPOINT ["/opt/batch-shipyard/shipyard.py"]

# https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-apt?view=azure-cli-latest#no-package-for-your-distribution
# ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $(lsb_release -sc) main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 

# https://github.com/Azure/azure-functions-core-tools
ENV AzFunctionsCoreTools_DIR /azure-functions-cli
RUN mkdir /tmp/AzFunctionsCoreTools3 && \
    mkdir -p $AzFunctionsCoreTools_DIR && \
    cd /tmp/AzFunctionsCoreTools3 && \
    wget --quiet --no-check-certificate \
    https://github.com/Azure/azure-functions-core-tools/releases/download/2.7.2100/Azure.Functions.Cli.linux-x64.2.7.2100.zip  && \
    unzip  -qq -d $AzFunctionsCoreTools_DIR Azure.Functions.Cli.linux-x64.*.zip && \
    rm -f Azure.Functions.Cli.linux-x64.*.zip && \
    cd $AzFunctionsCoreTools_DIR && \
    chmod +x func
ENV PATH=$AzFunctionsCoreTools_DIR:$PATH    

Overwriting /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito


In [10]:
cli_command='docker build -t '+ docker_image_name + \
' -f ' + docker_file_path + \
' ' + docker_build_dir + ' ' +\
docker_build_no_cache  + ' ' 
#  ' --target AzureBatchShipyardAzCliAzFunctionsCoreTools'

cli_command
# docker_build_response = ! $cli_command

# docker_build_response[0:5] 
# docker_build_response[-5:]

'docker build -t fwi01acr.azurecr.io/azure_batch_shipyard_devito -f /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito /workspace/notebooks/docker_build  '

In [11]:
! $cli_command

Sending build context to Docker daemon  3.631kB
Step 1/7 : FROM debian:latest
latest: Pulling from library/debian

Digest: sha256:d986a531d62903b66e731d475988f5b2ba3a4a90078078cb0f29f9685ee36466[1A
Status: Downloaded newer image for debian:latest
 ---> a8797652cfd9
Step 2/7 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in cdfd46306f22
Removing intermediate container cdfd46306f22
 ---> bf3e2ccf9c3b
Step 3/7 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     python3 python3-pip python3-setuptools     ca-certificates curl apt-transport-https lsb-release gnupg     zip unzip     wget rsync git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*   &&     git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard &&     cd /opt/batch-shipyard &&     rm -rf .git .github .vsts &&     rm -f .git* .travis.yml *.yml install*  &&     python3 -m pip install --no-cache-dir --upgrade pip  &&     pip3 install --no-cache-dir -r

Get:48 http://deb.debian.org/debian buster/main amd64 liberror-perl all 0.17027-2 [30.9 kB]
Get:49 http://deb.debian.org/debian buster/main amd64 git-man all 1:2.20.1-2+deb10u1 [1620 kB]
Get:50 http://deb.debian.org/debian buster/main amd64 git amd64 1:2.20.1-2+deb10u1 [5620 kB]
Get:51 http://deb.debian.org/debian buster/main amd64 gnupg-l10n all 2.2.12-1+deb10u1 [1010 kB]
Get:52 http://deb.debian.org/debian buster/main amd64 gnupg-utils amd64 2.2.12-1+deb10u1 [861 kB]
Get:53 http://deb.debian.org/debian buster/main amd64 gpg amd64 2.2.12-1+deb10u1 [865 kB]
Get:54 http://deb.debian.org/debian buster/main amd64 pinentry-curses amd64 1.1.0-2 [64.5 kB]
Get:55 http://deb.debian.org/debian buster/main amd64 gpg-agent amd64 2.2.12-1+deb10u1 [617 kB]
Get:56 http://deb.debian.org/debian buster/main amd64 gpg-wks-client amd64 2.2.12-1+deb10u1 [485 kB]
Get:57 http://deb.debian.org/debian buster/main amd64 gpg-wks-server amd64 2.2.12-1+deb10u1 [478 kB]
Get:58 http://deb.debian.org/debian buster/m

Selecting previously unselected package libldap-2.4-2:amd64.
Preparing to unpack .../15-libldap-2.4-2_2.4.47+dfsg-3+deb10u1_amd64.deb ...
Unpacking libldap-2.4-2:amd64 (2.4.47+dfsg-3+deb10u1) ...
Selecting previously unselected package libnghttp2-14:amd64.
Preparing to unpack .../16-libnghttp2-14_1.36.0-2+deb10u1_amd64.deb ...
Unpacking libnghttp2-14:amd64 (1.36.0-2+deb10u1) ...
Selecting previously unselected package librtmp1:amd64.
Preparing to unpack .../17-librtmp1_2.4+20151223.gitfa8646d.1-2_amd64.deb ...
Unpacking librtmp1:amd64 (2.4+20151223.gitfa8646d.1-2) ...
Selecting previously unselected package libssh2-1:amd64.
Preparing to unpack .../18-libssh2-1_1.8.0-2.1_amd64.deb ...
Unpacking libssh2-1:amd64 (1.8.0-2.1) ...
Selecting previously unselected package libcurl4:amd64.
Preparing to unpack .../19-libcurl4_7.64.0-4_amd64.deb ...
Unpacking libcurl4:amd64 (7.64.0-4) ...
Selecting previously unselected package curl.
Preparing to unpack .../20-curl_7.64.0-4_amd64.deb ...
Unpacking

Setting up libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3.7 (3.7.3-2+deb10u1) ...
Setting up gpg-agent (2.2.12-1+deb10u1) ...
Setting up gpgsm (2.2.12-1+deb10u1) ...
Setting up libcurl3-gnutls:amd64 (7.64.0-4) ...
Setting up python3 (3.7.3-1) ...
running python rtupdate hooks for python3.7...
running python post-rtupdate hooks for python3.7...
Setting up dirmngr (2.2.12-1+deb10u1) ...
Setting up perl (5.28.1-6) ...
Setting up gpg-wks-server (2.2.12-1+deb10u1) ...
Setting up lsb-release (10.2019051400) ...
Setting up python3-lib2to3 (3.7.3-1) ...
Setting up python3-pkg-resources (40.8.0-1) ...
Setting up python3-distutils (3.7.3-1) ...
Setting up python3-setuptools (40.8.0-1) ...
Setting up gpg-wks-client (2.2.12-1+deb10u1) ...
Setting up liberror-perl (0.17027-2) ...
Setting up git (1:2.20.1-2+deb10u1) ...
Setting up python3-pip (18.1-5) ...
Setting up gnupg (2.2.12-1+deb10u1) ...
Processing triggers for libc-bin (2.28-10) ...
Processing triggers for ca-certificates (20190110

Removing intermediate container 3b2a9da3643d
 ---> a92ed5a9eaa9
Step 4/7 : RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $(lsb_release -sc) main" |     tee /etc/apt/sources.list.d/azure-cli.list &&     curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - &&     apt-get update &&     apt-get install -y --no-install-recommends     azure-cli
 ---> Running in ac5ce1cbc869
deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ buster main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  44681      0 --:--:-- --:--:-- --:--:-- 44681
OK
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:3 https://packages.microsoft.com/repos/azure-cli buster InRelease [29.7 kB]
Get:4 http://security-cdn.debian.org/debian-securit

In [12]:
dotenv.load_dotenv(dotenv_path=sibling_docker_dotenv_file_path)
os.getenv('DOCKER_CONTAINER_MOUNT_POINT')

True

'/datadrive01/prj/Azure2019/notebooks'

In [13]:
cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v ~/.ssh:/root/.ssh:ro '+ \
'-v /usr/bin/docker:/usr/bin/docker '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
docker_image_name + ' '+ \
'/bin/bash -c ' 

# cli_command = cli_base_command+'"pwd; ls -l /workspace"'

image_name = azure_batch_shipyard_devito_base_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t '+\
image_name + ' ."'
cli_command = cli_base_command+sibling_docker_build_command
cli_command
!$cli_command

image_name = azure_batch_shipyard_devito_tti_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t '+\
image_name + ' ."'
cli_command = cli_base_command+sibling_docker_build_command
cli_command
!$cli_command


'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v ~/.ssh:/root/.ssh:ro -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock fwi01acr.azurecr.io/azure_batch_shipyard_devito /bin/bash -c "cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t devito_azure_base:v1.0 ."'

Sending build context to Docker daemon  4.096kB
Step 1/9 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu

9e3a4d10: Pulling fs layer 
19cdbe7a: Pulling fs layer 
61ea6baf: Pulling fs layer 
9d2d6c4f: Pull complete  162B/162B7kBBDigest: sha256:8d31dad0c58f552e890d68bbfb735588b6b820a46e459672d96e585871acc110
Status: Downloaded newer image for ubuntu:18.04
 ---> ccc6e87d482b
Step 2/9 : COPY ssh_config /root/.ssh/config
 ---> 692f75205b15
Step 3/9 : RUN apt-get update     && apt-get install -y --no-install-recommends         build-essential ca-certificates wget openssh-client openssh-server         mpich libmpich-dev     && mkdir -p /var/run/sshd     && ssh-keygen -A     && sed -i 's/UsePAM yes/UsePAM no/g' /etc/ssh/sshd_config     && sed -i 's/#PermitRootLogin yes/PermitRootLogin yes/g' /etc/ssh/sshd_config     && sed -i 's/#RSAAuthentication yes/RSAAuthentication yes/g' /etc/ssh/sshd_config     && sed -i 's/#PubkeyAuthentication yes/PubkeyAuthentication yes/g' /etc/ssh/sshd_config

Get:25 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 binutils-x86-64-linux-gnu amd64 2.30-21ubuntu1~18.04.2 [1856 kB]
Get:26 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 binutils amd64 2.30-21ubuntu1~18.04.2 [3396 B]
Get:27 http://archive.ubuntu.com/ubuntu bionic/main amd64 libc-dev-bin amd64 2.27-3ubuntu1 [71.8 kB]
Get:28 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 linux-libc-dev amd64 4.15.0-76.86 [999 kB]
Get:29 http://archive.ubuntu.com/ubuntu bionic/main amd64 libc6-dev amd64 2.27-3ubuntu1 [2587 kB]
Get:30 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc-7-base amd64 7.4.0-1ubuntu1~18.04.1 [18.9 kB]
Get:31 http://archive.ubuntu.com/ubuntu bionic/main amd64 libisl19 amd64 0.19-1 [551 kB]
Get:32 http://archive.ubuntu.com/ubuntu bionic/main amd64 libmpfr6 amd64 4.0.1-1 [243 kB]
Get:33 http://archive.ubuntu.com/ubuntu bionic/main amd64 libmpc3 amd64 1.1.0-1 [40.8 kB]
Get:34 http://archive.ubuntu.com/ubuntu bionic-updates/main amd

Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../18-libpsl5_0.19.1-5build1_amd64.deb ...
Unpacking libpsl5:amd64 (0.19.1-5build1) ...
Selecting previously unselected package libssl1.0.0:amd64.
Preparing to unpack .../19-libssl1.0.0_1.0.2n-1ubuntu5.3_amd64.deb ...
Unpacking libssl1.0.0:amd64 (1.0.2n-1ubuntu5.3) ...
Selecting previously unselected package openssh-client.
Preparing to unpack .../20-openssh-client_1%3a7.6p1-4ubuntu0.3_amd64.deb ...
Unpacking openssh-client (1:7.6p1-4ubuntu0.3) ...
Selecting previously unselected package wget.
Preparing to unpack .../21-wget_1.19.4-1ubuntu2.2_amd64.deb ...
Unpacking wget (1.19.4-1ubuntu2.2) ...
Selecting previously unselected package binutils-common:amd64.
Preparing to unpack .../22-binutils-common_2.30-21ubuntu1~18.04.2_amd64.deb ...
Unpacking binutils-common:amd64 (2.30-21ubuntu1~18.04.2) ...
Selecting previously unselected package libbinutils:amd64.
Preparing to unpack .../23-libbinutils_2.30-21ubuntu1~18.04

Selecting previously unselected package openssh-server.
Preparing to unpack .../66-openssh-server_1%3a7.6p1-4ubuntu0.3_amd64.deb ...
Unpacking openssh-server (1:7.6p1-4ubuntu0.3) ...
Selecting previously unselected package libhwloc5:amd64.
Preparing to unpack .../67-libhwloc5_1.11.9-1_amd64.deb ...
Unpacking libhwloc5:amd64 (1.11.9-1) ...
Selecting previously unselected package hwloc-nox.
Preparing to unpack .../68-hwloc-nox_1.11.9-1_amd64.deb ...
Unpacking hwloc-nox (1.11.9-1) ...
Selecting previously unselected package libmpich12:amd64.
Preparing to unpack .../69-libmpich12_3.3~a2-4_amd64.deb ...
Unpacking libmpich12:amd64 (3.3~a2-4) ...
Selecting previously unselected package libmpich-dev.
Preparing to unpack .../70-libmpich-dev_3.3~a2-4_amd64.deb ...
Unpacking libmpich-dev (3.3~a2-4) ...
Selecting previously unselected package mpich.
Preparing to unpack .../71-mpich_3.3~a2-4_amd64.deb ...
Unpacking mpich (3.3~a2-4) ...
Setting up libquadmath0:amd64 (8.3.0-6ubuntu1~18.04.1) ...
Sett

...

Selecting previously unselected package python3-pip.
Preparing to unpack .../32-python3-pip_9.0.1-2.3~ubuntu1.18.04.1_all.deb ...
Unpacking python3-pip (9.0.1-2.3~ubuntu1.18.04.1) ...
Selecting previously unselected package python3-pkg-resources.
Preparing to unpack .../33-python3-pkg-resources_39.0.1-2_all.deb ...
Unpacking python3-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python3-setuptools.
Preparing to unpack .../34-python3-setuptools_39.0.1-2_all.deb ...
Unpacking python3-setuptools (39.0.1-2) ...
Selecting previously unselected package python3-wheel.
Preparing to unpack .../35-python3-wheel_0.30.0-0.2_all.deb ...
Unpacking python3-wheel (0.30.0-0.2) ...
Selecting previously unselected package python3-xdg.
Preparing to unpack .../36-python3-xdg_0.25-4ubuntu1_all.deb ...
Unpacking python3-xdg (0.25-4ubuntu1) ...
Setting up libpython3-stdlib:amd64 (3.6.7-1~18.04) ...
Setting up python-pip-whl (9.0.1-2.3~ubuntu1.18.04.1) ...
Setting up libicu60:amd64 (60.2-

  Created wheel for devito: filename=devito-4.0+393.g6244c921-py3-none-any.whl size=373099 sha256=0b7af86adc2e8bb5e6476ca1dd7127b6d73654609c259226f746a9aacea5301e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5siipqi3/wheels/fa/16/d5/e6ff4407842c8c261bdbc5a422361f1287c005fbb1f847798e
  Created wheel for cgen: filename=cgen-2020.1-py3-none-any.whl size=16549 sha256=be62109af68bdb9462cc13b67d9326f35222049c39516b109292e2b174c683a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-5siipqi3/wheels/a9/dd/e4/6df7db32de13ab516b2a1e4062ddfc4e6dea522811bc58a2ee
  Created wheel for codepy: filename=codepy-2019.1-py3-none-any.whl size=20626 sha256=27a0924fd3cf63316ce21e7d1547af7b70d9dbdd70ef8adfcb9651fba84fefb2
  Stored in directory: /tmp/pip-ephem-wheel-cache-5siipqi3/wheels/e1/48/1f/4840360a258052536e68084f4acd3ca14ea7bb9a3a08fd12da
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3580 sha256=6193c880fca3b25f30b99a0d64d5611d8aeba912364d49b0c012d6a749ebcfc5
  Stor

  Created wheel for backcall: filename=backcall-0.1.0-py3-none-any.whl size=11928 sha256=b5666a91e9cdc552757ab402f81a5943bc41ad00c7bc84f376e6ad1419fd5a4e
  Stored in directory: /root/.cache/pip/wheels/b4/cb/f1/d142b3bb45d488612cf3943d8a1db090eb95e6687045ba61d1
Successfully built devito cgen codepy frozendict psutil py-cpuinfo pyrevolve pytools tornado pyyaml toolz contexttimer mpmath pyrsistent backcall
Removing intermediate container 468245e7f574
 ---> 37f1128b63f5
Step 8/9 : RUN pip3 install mpi4py matplotlib
 ---> Running in 1dd0ad8cd18b
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2073285 sha256=48674b4626f60e56855fc50988d3d5e87f48a80afc0f1ced03e3987b52f6ac12
  Stored in directory: /root/.cache/pip/wheels/d6/73/83/ad9dd3ebae512829ab3f21657f76403dc4aa6649e1118c9369
Successfully built mpi4py
Removing intermediate container 1dd0ad8cd18b
 ---> 6dca13ef736c
Step 9/9 : RUN rm -rf /var/lib/apt/lists/* 	&& apt-get clean
 ---> Running in 0ea7bf73db71
Re

'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v ~/.ssh:/root/.ssh:ro -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock fwi01acr.azurecr.io/azure_batch_shipyard_devito /bin/bash -c "cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t devito_azure_tti:v1.0 ."'

Sending build context to Docker daemon  84.99kB
Step 1/6 : FROM devito_azure_base:v1.0
 ---> f0fc0189866d
Step 2/6 : RUN pip3 install azure-storage-blob h5py segyio boto3
 ---> Running in 5a8248af9f88
Removing intermediate container 5a8248af9f88
 ---> 401999814792
Step 3/6 : RUN pip3 install azure-storage
 ---> Running in 57b9e204f5e7
Removing intermediate container 57b9e204f5e7
 ---> 0e7752563fc2
Step 4/6 : ADD ./tti /app/tti
 ---> 5d5eb4ae416a
Step 5/6 : EXPOSE 23
 ---> Running in 57f8782e2a2d
Removing intermediate container 57f8782e2a2d
 ---> b701591ff744
Step 6/6 : CMD ["/usr/sbin/sshd", "-D", "-p", "23"]
 ---> Running in 82ea8ebdf508
Removing intermediate container 82ea8ebdf508
 ---> 26d633848a3d
Successfully built 26d633848a3d
Successfully tagged devito_azure_tti:v1.0


##### Use control plane docker image to self push into ACR

In [ ]:
full_azure_batch_shipyard_devito_base_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_base_image_name
full_azure_batch_shipyard_devito_tti_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_tti_image_name

internal_command = '"'+ \
' az login; '+ \
'az account set --subscription '+os.getenv('SUBSCRIPTION_ID')+ '; '+ \
' az account list -o table; '+ \
'az acr login --name '+os.getenv('ACR_NAME')+ \
' --username '+os.getenv('ACR_USERNAME')+ \
' --password ' + os.getenv('ACR_PASSWORD')+'; '+\
'docker push '+docker_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_base_image_name+' '+ full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_tti_image_name+' '+ full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'"'
cli_command = cli_base_command+internal_command
cli_command
!$cli_command

In [15]:
if use_ACR_and_dockerhub:
    control_plane_docker_image_dockerhub_name = os.getenv('DOCKERHUB_LOGIN')+ '/' + docker_image_name_only
    cli_command='docker tag '+ docker_image_name+' '+ control_plane_docker_image_dockerhub_name
    cli_command

    docker_pwd = os.getenv('DOCKERHUB_PWD')
    docker_login = os.getenv('DOCKERHUB_LOGIN')
    !docker login -u=$docker_login -p=$docker_pwd
    !docker push {control_plane_docker_image_dockerhub_name}
 

'docker tag fwi01acr.azurecr.io/azure_batch_shipyard_devito georgedockeraccount/azure_batch_shipyard_devito'

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [docker.io/georgedockeraccount/azure_batch_shipyard_devito]
An image does not exist locally with the tag: georgedockeraccount/azure_batch_shipyard_devito
